## Pasos realizados:


En primer lugar, se descargaron los datos sobre estimación de abortos realizados por la World Health Organization disponibles en [este enlace](https://www.who.int/data/gho/data/indicators/indicator-details/GHO/SRH_ABORTION_RATE). Una vez descargados, se suben a colab de la siguiente forma:



In [6]:
from google.colab import files
uploaded = files.upload()

Saving data.csv to data (2).csv


 Se importa pandas y se crea un nuevo dataframe con estos datos.

In [7]:
import pandas as pd

nombre_archivo = 'data.csv'
df = pd.read_csv(nombre_archivo, sep=';')
df.head()


,IndicatorCode,Indicator,ValueType,ParentLocationCode,ParentLocation,Location type,SpatialDimValueCode,Location,Period type,Period,...,FactValueUoM,FactValueNumericLowPrefix,FactValueNumericLow,FactValueNumericHighPrefix,FactValueNumericHigh,Value,FactValueTranslationID,FactComments,Language,DateModified
0,SRH_ABORTION_RATE,Abortion rate (model-estimated),numeric,EUR,Europe,Country,POL,Poland,Year,2015-2019,...,NaN,NaN,4,NaN,28,10 [4-28],NaN,NaN,EN,26-04-2023 0:00
1,SRH_ABORTION_RATE,Abortion rate (model-estimated),numeric,EUR,Europe,Country,POL,Poland,Year,2015-2019,...,NaN,NaN,6,NaN,20,10 [6-20],NaN,NaN,EN,26-04-2023 0:00
2,SRH_ABORTION_RATE,Abortion rate (model-estimated),numeric,EUR,Europe,Country,BGR,Bulgaria,Year,2015-2019,...,NaN,NaN,9,NaN,11,10 [9-11],NaN,NaN,EN,26-04-2023 0:00
3,SRH_ABORTION_RATE,Abortion rate (model-estimated),numeric,EUR,Europe,Country,LVA,Latvia,Year,2015-2019,...,NaN,NaN,9,NaN,11,10 [9-11],NaN,NaN,EN,26-04-2023 0:00
4,SRH_ABORTION_RATE,Abortion rate (model-estimated),numeric,EUR,Europe,Country,BLR,Belarus,Year,2015-2019,...,NaN,NaN,9,NaN,12,10 [9-12],NaN,NaN,EN,26-04-2023 0:00


A continuación se procede con la limpieza de los datos. En primer lugar, eliminamos todas las columnas en las cuales se repite el mismo valor para todas las filas y aquellas para las cuales todos los valores son nulos:

In [8]:
df = df.loc[:, (df != df.iloc[0]).any()]
df = df.dropna(axis=1, how='all')
df

,ParentLocationCode,ParentLocation,SpatialDimValueCode,Location,Dim1,Dim1ValueCode,FactValueNumeric,FactValueNumericLow,FactValueNumericHigh,Value
0,EUR,Europe,POL,Poland,95% uncertainty,UNCERTAINTY_INTERVAL_UI95,1046,4,28,10 [4-28]
1,EUR,Europe,POL,Poland,80% uncertainty,UNCERTAINTY_INTERVAL_UI80,1046,6,20,10 [6-20]
2,EUR,Europe,BGR,Bulgaria,80% uncertainty,UNCERTAINTY_INTERVAL_UI80,1023,9,11,10 [9-11]
3,EUR,Europe,LVA,Latvia,80% uncertainty,UNCERTAINTY_INTERVAL_UI80,1027,9,11,10 [9-11]
4,EUR,Europe,BLR,Belarus,80% uncertainty,UNCERTAINTY_INTERVAL_UI80,105,9,12,10 [9-12]
...,...,...,...,...,...,...,...,...,...,...
283,EUR,Europe,GEO,Georgia,80% uncertainty,UNCERTAINTY_INTERVAL_UI80,803,55,113,80 [55-113]
284,EUR,Europe,ALB,Albania,95% uncertainty,UNCERTAINTY_INTERVAL_UI95,945,5,18,9 [5-18]
285,EUR,Europe,ALB,Albania,80% uncertainty,UNCERTAINTY_INTERVAL_UI80,945,6,14,9 [6-14]
286,EUR,Europe,ESP,Spain,80% uncertainty,UNCERTAINTY_INTERVAL_UI80,873,8,9,9 [8-9]


Además, para cada país tenemos dos filas, ya que se realizaron estimaciones al 80% y 95% de incertidumbre. Queremos que para cada país solamente quede una fila:

In [9]:
df_80 = df[df['Dim1ValueCode'] == 'UNCERTAINTY_INTERVAL_UI80'].copy()
df_95 = df[df['Dim1ValueCode'] == 'UNCERTAINTY_INTERVAL_UI95'].copy()

df_80 = df_80.rename(columns={
    'FactValueNumericLow': 'FactValueNumericLow_80',
    'FactValueNumericHigh': 'FactValueNumericHigh_80',
    'Value': 'Value_80'
})

df_95 = df_95.rename(columns={
    'FactValueNumericLow': 'FactValueNumericLow_95',
    'FactValueNumericHigh': 'FactValueNumericHigh_95',
    'Value': 'Value_95'
})
df_80 = df_80[['ParentLocation', 'SpatialDimValueCode', 'Location', 'FactValueNumericLow_80', 'FactValueNumericHigh_80', 'Value_80']]
df_95 = df_95[['ParentLocation', 'SpatialDimValueCode', 'Location', 'FactValueNumericLow_95', 'FactValueNumericHigh_95', 'Value_95']]

df_combinado = pd.merge(df_80, df_95, on=['ParentLocation', 'SpatialDimValueCode', 'Location'], how='inner')
df_combinado.head()


,ParentLocation,SpatialDimValueCode,Location,FactValueNumericLow_80,FactValueNumericHigh_80,Value_80,FactValueNumericLow_95,FactValueNumericHigh_95,Value_95
0,Europe,POL,Poland,6,20,10 [6-20],4,28,10 [4-28]
1,Europe,BGR,Bulgaria,9,11,10 [9-11],9,12,10 [9-12]
2,Europe,LVA,Latvia,9,11,10 [9-11],9,12,10 [9-12]
3,Europe,BLR,Belarus,9,12,10 [9-12],9,12,10 [9-12]
4,Europe,NOR,Norway,11,11,11 [11-11],10,11,11 [10-11]


Finalmente, limpiamos las columnas "Value_80" y "Value_95", ya que estas incluyen también los intervalos de incertidumbre y esa información ya se encuentra en otras columnas.

In [10]:
df_combinado['Value_80'] = df_combinado['Value_80'].astype(str).str.split(' ').str[0]
df_combinado['Value_95'] = df_combinado['Value_95'].astype(str).str.split(' ').str[0]
df_combinado.head()

,ParentLocation,SpatialDimValueCode,Location,FactValueNumericLow_80,FactValueNumericHigh_80,Value_80,FactValueNumericLow_95,FactValueNumericHigh_95,Value_95
0,Europe,POL,Poland,6,20,10,4,28,10
1,Europe,BGR,Bulgaria,9,11,10,9,12,10
2,Europe,LVA,Latvia,9,11,10,9,12,10
3,Europe,BLR,Belarus,9,12,10,9,12,10
4,Europe,NOR,Norway,11,11,11,10,11,11


Ya estamos listos para exportar los datos como un nuevo archivo .csv

In [11]:
df_combinado.to_csv('Estimación_Tasa_de_Abortos.csv', index=False)
files.download('Estimación_Tasa_de_Abortos.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>